In [1]:
#imports
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import csv
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
import torch.nn.functional as F

#Provided Eval Metrics

#Preprocessing

In [130]:

# data load with tfidf
train_data = pd.read_csv("train_data.csv")

x=train_data['utterances']
y = [label.split() for label in train_data['Core Relations']]

binarizer = MultiLabelBinarizer()
binarizer.fit(y)
y = binarizer.transform(y)

x_train, x_dt, y_train, y_dt = train_test_split(x, y, test_size = .1, random_state = 0)
#x_dev, x_test, y_dev, y_test = train_test_split(x_dt, y_dt, test_dt = .5, random_state = 0)
tfidf = TfidfVectorizer(min_df = 0)
fitted = tfidf.fit(x_train.append(x_dt))
x_train = fitted.transform(x_train)
x_dt = fitted.transform(x_dt)
print(x_train.shape)
train_data.describe()

(1577, 1117)


,utterances,Core Relations
count,2253,2253
unique,2163,47
top,who directed finding nemo,movie.directed_by
freq,6,314


In [2]:
#glove download
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r', encoding="utf-8") as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector
train_data = pd.read_csv("train_data.csv")


In [113]:
#data load with word embedding and padding

#def find_closest_embeddings(embedding):
#    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))


x=[label.split() for label in train_data['utterances']]
#y = [label.split() for label in train_data['Core Relations']]
y = train_data['Core Relations']
#get vocab
target_vocab = {}
index = 0
for sentence in x:
  for word in sentence:
    if word in target_vocab.keys():
      continue
    else:
      target_vocab[word] = index
      index = index +1

#embeddings
matrix_len = len(target_vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(target_vocab):
    try: 
        weights_matrix[i] = embeddings_dict[word]
        words_found += 1
    except KeyError:
       continue
        #weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

#labels
tag_vocab = {}
index = 0
for i in y:
  for word in y:
    if word in tag_vocab.keys():
      continue
    else:
      tag_vocab[word] = index
      index = index +1
#binarizer = MultiLabelBinarizer()
#binarizer.fit(y)
#y = binarizer.transform(y)



x_train, x_dt, y_train, y_dt = train_test_split(x, y, test_size = .1, random_state = 0)
print (type(weights_matrix))
EMBEDDING_DIM = 50
HIDDEN_DIM = 50
print (tag_vocab)
print (len(tag_vocab))
print (x_train[0], y_train[0])


<class 'numpy.ndarray'>
{'movie.starring.actor movie.starring.character': 0, 'movie.starring.actor': 1, 'movie.starring.actor actor.gender': 2, 'person.date_of_birth': 3, 'movie.estimated_budget': 4, 'movie.estimated_budget movie.directed_by': 5, 'movie.starring.character': 6, 'movie.music': 7, 'movie.initial_release_date': 8, 'movie.locations': 9, 'movie.locations movie.directed_by': 10, 'movie.produced_by': 11, 'movie.production_companies': 12, 'movie.directed_by': 13, 'movie.starring.actor movie.directed_by': 14, 'movie.country': 15, 'movie.directed_by movie.initial_release_date': 16, 'movie.production_companies movie.initial_release_date': 17, 'movie.starring.actor movie.initial_release_date': 18, 'movie.directed_by movie.produced_by': 19, 'movie.directed_by movie.subjects': 20, 'movie.directed_by movie.genre': 21, 'movie.genre': 22, 'movie.production_companies movie.genre': 23, 'movie.language': 24, 'none': 25, 'movie.country movie.language movie.genre': 26, 'movie.country movie.l

#Models

In [ ]:
#single direction lstm
#for tfidf (1577, 1117)
def prepare_sequence(seq, to_ix):
    #return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)
def prepare_sequenceT(seq, to_ix):
    #return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]
    idxs = to_ix[seq]
    #print (seq)
    #print (idxs)
    #print (to_ix)
    return torch.tensor([idxs], dtype=torch.long)

def create_emb_layer(weights_matrix, non_trainable=False):
    #modified check numbers are right
    num_embeddings= len(weights_matrix)
    embedding_dim = len(weights_matrix[1])

    emb_layer = torch.nn.Embedding(num_embeddings, embedding_dim)
    #important step that doesnt work fix this
    weights_matrix = torch.tensor(weights_matrix, dtype=torch.long)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, weights_matrix):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        #modify this
        #self.embeddings, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        #self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(weights_matrix))
        self.embeddings.weight.requires_grad = False
        self.dropout = torch.nn.Dropout(0.3)
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)
        self.linear = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence, s):
        x = self.embeddings(sentence)
        print (x.shape)
        #switch point for an RNN or GRU
        #self.RNN(x)
        #self.GRU(x.view(len(sentence), 1, -1))
        lstm_out, (ht, ct) = self.lstm(x.view(len(sentence), 1, -1))
        print (lstm_out.shape)
        lin =self.linear(ht[-1])
        print(lin.shape)
        #tag_scores = F.log_softmax(lin)
        return lin



model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(target_vocab), len(tag_vocab), weights_matrix)
loss_function = F.cross_entropy
#torch.nn.MSELoss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = torch.optim.Adam([var1, var2], lr=0.0001)
#adagrad RMSprop implementing SGD vs Adam this time

for epoch in range(100):  
    sentences = []
    targets = []
    for i in range(0, len(x_train)):
        sentences.append(prepare_sequence(x_train[i], target_vocab))
        #print (y_train.to_list())
        targets.append(prepare_sequenceT(y_train.to_list()[i], tag_vocab))
        #targets.append(torch.FloatTensor((y_train)))
    torch.nn.utils.rnn.pad_sequence(sentences)
    #torch.nn.utils.rnn.pad_sequence(targets)
    for i  in range(0, len(x_train)):
        #  clear gradients aparently
        model.zero_grad()
        #tensors
        sentence_in = sentences[i]
        target = targets[i]
        #print (targets)
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in, 40)

        #computing loss
        #print (len(tag_scores))

        print(tag_scores)
        print (target)
        loss = loss_function(tag_scores, target)
        loss.backward()
        optimizer.step()

In [ ]:
#verifying lstm results
y_pred = []
count = 0
right = 0
with torch.no_grad():
  for i in range(0, len(y_dt)):
    pred = (np.argmax(model(prepare_sequence(x_train[i], target_vocab), 40).numpy()))
    y_pred.append([key for key, value in tag_vocab.items() if value == pred]
)
    print (y_pred[i])
  for i in range(0,len(y_pred)):
    if y_pred[i] == ():
      y_pred[i] = ('none',)
    if y_pred[i] == y_dt[i]:
      right = right +1
    #else:
    #  print (y_pred[i], y_dt[i])
    count = count + 1
  ex = 3
  print (y_pred[ex], y_dt[ex])
  print (right)
  print (count)

In [131]:
#default approach 
classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha =.0001))
classifier.fit(x_train, y_train)


y_pred = classifier.predict(x_dt)
count = 0
right = 0
y_pred = binarizer.inverse_transform(y_pred)
y_dt = binarizer.inverse_transform(y_dt)
for i in range(0,len(y_pred)):
  if y_pred[i] == ():
    y_pred[i] = ('none',)
  if y_pred[i] == y_dt[i]:
    right = right +1
  #else:
  #  print (y_pred[i], y_dt[i])
  count = count + 1
ex = 2
print (y_pred[ex], y_dt[ex])
print (right)
print (count)

('movie.directed_by',) ('movie.directed_by',)
539
676


In [132]:
ex = 423
print (y_pred[ex], y_dt[ex])

('movie.initial_release_date', 'movie.production_companies') ('movie.initial_release_date', 'movie.production_companies')
